<a href="https://colab.research.google.com/github/aliyyah-u/NLP_Medical_NER/blob/main/NLP_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
__author__ = "Aliyyah U."
__version__ = "IN3045 City St George's, University of London, Spring 2025"

In [ ]:
!pip install --upgrade huggingface_hub

from huggingface_hub import login
login()
# Login using e.g. `huggingface-cli login` to access this dataset

import pandas as pd

df = pd.read_parquet("hf://datasets/parsa-mhmdi/Medical_NER/data/train-00000-of-00001.parquet")

df[:3]